In [8]:
import torch
import os
from pathlib import Path
from torch.utils.tensorboard import SummaryWriter
from monai.utils import first, set_determinism
from monai.transforms import (
    EnsureChannelFirstd,
    Compose,
    CropForegroundd,
    LoadImaged,
    Orientationd,
    ScaleIntensityRanged,
    Spacingd,
    RandFlipd,
    Resized,
    NormalizeIntensityd,
    ResizeWithPadOrCropd,
    RandSpatialCropd
)
from monai.data import DataLoader, Dataset, ImageDataset
from monai.config import print_config
from monai.apps import download_and_extract
from monai.visualize import blend_images, matshow3d, plot_2d_or_3d_image
from load_data import get_data
from transforms import transforms

In [9]:
set_determinism(seed=2023)
home = Path.home().as_posix()

In [10]:
datasets = {
    "ixi": {"path": home + r"/datasets/ixi/ixi_dataset.json", "label": "sex"},
    "ukb": {"path": home + r"/datasets/ukb/ukb_dataset.json", "label": "sex"},
}
    
data = get_data(
    "ixi",
    label=datasets["ixi"]["label"],
)

data["train"] = [{"image": os.path.join(home, i["image"]), "label": i["label"]} for i in data["train"]]
#print(data["train"])
#data_dicts = [{"image": files[i], "label": data["train"][1][i]} for i in range(len(files))]

transform = Compose(
    [
        LoadImaged(keys=["image"]),
        EnsureChannelFirstd(keys=["image"]),
        #RandFlipd(keys=["image"], prob=1, spatial_axis=0),
        #Resized(keys=["image"], spatial_size=(40, 40, 40)),
        Spacingd(keys=['image'], pixdim=2, mode=("bilinear")),
        ResizeWithPadOrCropd(keys=['image'], spatial_size=(90+5, 90+5, 65+5), mode='constant', value=0.0),
        RandSpatialCropd(keys=['image'], roi_size=[90, 90, 65], random_center=True, random_size=False),
        NormalizeIntensityd(keys=["image"]),
        #Orientationd(keys=["image", "label"], axcodes="PLS"),
        #Spacingd(keys=["image", "label"], pixdim=(1.5, 1.5, 2.0), mode=("bilinear", "nearest")),
        #ScaleIntensityRanged(keys=["image"], a_min=-57, a_max=164, b_min=0.0, b_max=1.0, clip=True),
        #CropForegroundd(keys=["image", "label"], source_key="image"),
    ]
)

#t = LoadImaged(keys=["image"])(data[0])
#print(t)

train_dataset = Dataset(
    data=data["train"],
    transform=transform,
)

train_loader = DataLoader(
    train_dataset,
)

data_first = first(train_loader)
print(f"image shape: {data_first['image'].shape}, label shape: {data_first['label'].shape}")

figsize=(100, 100)

/home/neutron/.local/lib/python3.8/site-packages/monai/utils/deprecate_utils.py:321: FutureWarning: monai.transforms.io.dictionary LoadImaged.__init__:image_only: Current default value of argument `image_only=False` has been deprecated since version 1.1. It will be changed to `image_only=True` in version 1.3.
  warn_deprecated(argname, msg, warning_category)


image shape: torch.Size([1, 1, 90, 90, 65]), label shape: torch.Size([1])


In [11]:
matshow3d(
    volume=data_first["image"],
    fig=None,
    title="input image",
    figsize=figsize,
    every_n=10,
    frame_dim=-1,
    show=True,
    cmap="gray",
)

(<Figure size 10000x10000 with 1 Axes>,
 array([[       nan,        nan,        nan, ...,        nan,        nan,
                nan],
        [       nan, -0.6387539, -0.6387539, ..., -0.6387539, -0.6387539,
                nan],
        [       nan, -0.6387539, -0.6387539, ..., -0.6387539, -0.6387539,
                nan],
        ...,
        [       nan, -0.6387539, -0.6387539, ...,        nan,        nan,
                nan],
        [       nan, -0.6387539, -0.6387539, ...,        nan,        nan,
                nan],
        [       nan,        nan,        nan, ...,        nan,        nan,
                nan]], dtype=float32))

In [12]:
matshow3d(
    volume=data_first["image"],
    fig=None,
    title="input image",
    figsize=figsize,
    every_n=10,
    frame_dim=-2,
    show=True,
    cmap="gray",
)

(<Figure size 10000x10000 with 1 Axes>,
 array([[       nan,        nan,        nan, ...,        nan,        nan,
                nan],
        [       nan, -0.6387539, -0.6387539, ..., -0.6387539, -0.6387539,
                nan],
        [       nan, -0.6387539, -0.6387539, ..., -0.6387539, -0.6387539,
                nan],
        ...,
        [       nan, -0.6387539, -0.6387539, ..., -0.6387539, -0.6387539,
                nan],
        [       nan, -0.6387539, -0.6387539, ..., -0.6387539, -0.6387539,
                nan],
        [       nan,        nan,        nan, ...,        nan,        nan,
                nan]], dtype=float32))

In [13]:
matshow3d(
    volume=data_first["image"],
    fig=None,
    title="input image",
    figsize=figsize,
    every_n=10,
    frame_dim=-3,
    show=True,
    cmap="gray",
)

(<Figure size 10000x10000 with 1 Axes>,
 array([[       nan,        nan,        nan, ...,        nan,        nan,
                nan],
        [       nan, -0.6387539, -0.6387539, ..., -0.6387539, -0.6387539,
                nan],
        [       nan, -0.6387539, -0.6387539, ..., -0.6387539, -0.6387539,
                nan],
        ...,
        [       nan, -0.6387539, -0.6387539, ..., -0.6387539, -0.6387539,
                nan],
        [       nan, -0.6387539, -0.6387539, ..., -0.6387539, -0.6387539,
                nan],
        [       nan,        nan,        nan, ...,        nan,        nan,
                nan]], dtype=float32))